In [48]:
from gurobipy import *
from prettytable import PrettyTable
from numpy import array, concatenate,append

EC = Model('Multi Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100, 'd':2 ,'e':-5 ,'f':3 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'd':3.82 ,'e':-4.24 ,'f':6.09 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'd':5.01 ,'e':-2.15 ,'f':5.69 ,'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'd':1.1 ,'e':-3.99 ,'f':6.2 ,'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'd':3.55 ,'e':-6.88 ,'f':5.57 ,'Pmin':19 ,'Pmax':53 }  }

Iter =20
Load=400
TotalLoad=[]
Objective=[]
Var=[]
CostPeriod = []
EmissionPeriod = []

P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')
GenTotal = quicksum(P[i] for i in Gen)
GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)
GenEmissions = quicksum(GenAt[i]['f']+GenAt[i]['e']*P[i]+GenAt[i]['d']*P[i]*P[i] for i in Gen)

#Constraint
target = EC.addConstr( GenTotal == Load, name = 'DemandConst')

EC.setObjective(GenCost, GRB.MINIMIZE)

EC.Params.Method=-1
EC.setParam( 'OutputFlag', False )
EC.optimize()
CostMin = GenCost.getValue()
EmissionMax = GenEmissions.getValue()


EC.setObjective(GenEmissions, GRB.MINIMIZE)

EC.Params.Method=-1
EC.setParam( 'OutputFlag', False )
EC.optimize()
CostMax = GenCost.getValue()
EmissionMin = GenEmissions.getValue()


Results = PrettyTable(['Min Objective','Cost (€)','Emissions (kg)'])
Results.add_row(['Cost', round(CostMin,2), round(EmissionMax,2)])
Results.add_row(['Emissions', round(CostMax,2), round(EmissionMin,2)])                 
print(Results)




for c in range(Iter):
    ELim = (EmissionMin) + ((c)/(Iter-1))*(EmissionMax - EmissionMin)
    
    P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')
    GenTotal = quicksum(P[i] for i in Gen)
    GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)
    GenEmissions = quicksum(GenAt[i]['f']+GenAt[i]['e']*P[i]+GenAt[i]['d']*P[i]*P[i] for i in Gen)

    #Constraint
    target = EC.addConstr( GenTotal == Load, name = 'DemandConst')
    
    
    
    
    #Constraint    
    EmissionConstr = EC.addConstr( GenEmissions <= ELim, name = 'EmissionConstr')
    EC.setObjective(GenCost, GRB.MINIMIZE)

    EC.Params.Method=-1
    EC.setParam( 'OutputFlag', False )
    EC.optimize()
    
    CostMinI = GenCost.getValue()
    EmissionMinI = GenEmissions.getValue()
    CostPeriod.append(CostMinI)
    EmissionPeriod.append(EmissionMinI)
    

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
+---------------+-----------+----------------+
| Min Objective |  Cost (€) | Emissions (kg) |
+---------------+-----------+----------------+
|      Cost     |  131455.0 |    96450.75    |
|   Emissions   | 148684.73 |    87089.4     |
+---------------+-----------+----------------+


In [79]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from pandas import DataFrame
CP = array(CostPeriod, ndmin =2 )
EP = array(EmissionPeriod, ndmin = 2)
data = concatenate([CP.T, EP.T], axis =1)
print(data)
sns.scatterplot(x= 'cost (€)', y = 'Emissions (kg)', data=df )

[[147839.34143909  87093.51597442]
 [140519.04512634  87582.10602711]
 [137811.55710506  88074.73544455]
 [135998.0599212   88567.7317424 ]
 [134686.00458013  89060.22109313]
 [133782.44633373  89552.9068535 ]
 [133092.24213896  90089.56414757]
 [132582.28562291  90660.32325608]
 [132321.4935968   91075.10018413]
 [132119.39078559  91523.86355233]
 [131973.32120881  92016.52796221]
 [131856.01036873  92506.743254  ]
 [131755.57026302  92998.94576525]
 [131670.76222992  93494.70889841]
 [131601.55723747  93989.13160917]
 [131547.18165785  94480.0162894 ]
 [131505.83737178  94972.00096194]
 [131477.14435702  95465.38329372]
 [131460.43388359  95958.05450284]
 [131454.955673    96450.71439621]]


ValueError: Could not interpret input 'cost (€)'

In [78]:
from pandas import DataFrame
df = DataFrame([CostPeriod, EmissionPeriod])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,147839.341439,140519.045126,137811.557105,135998.059921,134686.004580,133782.446334,133092.242139,132582.285623,132321.493597,132119.390786,131973.321209,131856.010369,131755.570263,131670.762230,131601.557237,131547.181658,131505.837372,131477.144357,131460.433884,131454.955673
1,87093.515974,87582.106027,88074.735445,88567.731742,89060.221093,89552.906853,90089.564148,90660.323256,91075.100184,91523.863552,92016.527962,92506.743254,92998.945765,93494.708898,93989.131609,94480.016289,94972.000962,95465.383294,95958.054503,96450.714396


,87093.51597442251,87582.1060271132,88074.73544455203,88567.73174240079,89060.22109312988,89552.90685349966,90089.56414756842,90660.3232560769,91075.10018412975,91523.86355233223,92016.52796221219,92506.74325399671,92998.94576525084,93494.70889841249,93989.13160917035,94480.01628939825,94972.00096193964,95465.38329371612,95958.05450283804,96450.71439620633
147839.341439,,,,,,,,,,,,,,,,,,,,
140519.045126,,,,,,,,,,,,,,,,,,,,
137811.557105,,,,,,,,,,,,,,,,,,,,
135998.059921,,,,,,,,,,,,,,,,,,,,
134686.004580,,,,,,,,,,,,,,,,,,,,
133782.446334,,,,,,,,,,,,,,,,,,,,
133092.242139,,,,,,,,,,,,,,,,,,,,
132582.285623,,,,,,,,,,,,,,,,,,,,
132321.493597,,,,,,,,,,,,,,,,,,,,
132119.390786,,,,,,,,,,,,,,,,,,,,
